# nlp-project-vqa


In [ ]:
import moxing as mox
mox.file.copy_parallel(src_url="s3://nlp-haofeng/nlp_project_vqa", dst_url='./') 

In [1]:
import mindspore
import numpy as np
import os
from easydict import EasyDict
from preprocess.preprocess import *
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  # 允许重复载入lib文件

In [2]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE)

In [2]:
from mindspore import context
context.set_context(mode=context.PYNATIVE_MODE)

In [27]:
context.get_context('device_target') != 'CPU'

False

## 1 预处理

### 1.1 预处理配置

In [3]:
padding = '<pad>'
config = EasyDict({
	'train_ans_path': './data/annotations/train.json',
	'train_que_path': './data/questions/train.json',
	'valid_ans_path': './data/annotations/val.json',
	'valid_que_path': './data/questions/val.json',
	'test_ans_path':  './data/annotations/test.json',
	'test_que_path':  './data/questions/test.json',
	'train_img_path': './data/images/train/COCO_train2014_',
	'test_img_path': './data/images/test/COCO_val2014_',
	'val_img_path': './data/images/val/COCO_val2014_',
	'max_length': 25,
	'dict_path': './mindrecord/dict.npy',
	'idx_word_dict_path': './mindrecord/idx_word_dict.npy',
	'num_splits': 1,
	'train_mindrecord_path': './mindrecord/train.mindrecord',
	'valid_mindrecord_path': './mindrecord/valid.mindrecord',
	'test_mindrecord_path':  './mindrecord/test.mindrecord',
})

### 1.2 读取数据

注: 只取那些答案长度为1的vqa组合

In [4]:
# get 3 types of input data
train_images, train_questions, train_answers = get_list(config.train_que_path, config.train_ans_path)
valid_images, valid_questions, valid_answers = get_list(config.valid_que_path, config.valid_ans_path)
test_images,  test_questions,  test_answers  = get_list(config.test_que_path,  config.test_ans_path)

In [5]:
total_questions = train_questions + valid_questions + test_questions
total_answers = train_answers + valid_answers + test_answers

### 1.3 构建词典

In [6]:
# build word vocab
word_dict = dict({'<pad>': 0})
word_dict = add_word_into_dict(total_questions, word_dict)
word_dict = add_word_into_dict(total_answers, word_dict)

In [7]:
# build revert dict
idx_word_dict = dict()
for item in word_dict.items():
	idx_word_dict[item[1]] = item[0]

In [8]:
# save dict
np.save(config.dict_path, word_dict)
np.save(config.idx_word_dict_path, idx_word_dict)

### 1.4 向量化 & 补齐长度

In [9]:
# word -> vector & padding
train_questions_vec = get_vec_and_pad(train_questions, word_dict, config.max_length)
valid_questions_vec = get_vec_and_pad(valid_questions, word_dict, config.max_length)
test_questions_vec = get_vec_and_pad(test_questions, word_dict, config.max_length)

train_answers_vec = get_vec_and_pad(train_answers, word_dict, 1)
valid_answers_vec = get_vec_and_pad(valid_answers, word_dict, 1)
test_answers_vec = get_vec_and_pad(test_answers, word_dict, 1)


train_images_list = read_image(train_images, config.train_img_path)
np.save('./mindrecord/train_images_list', train_images_list)

valid_images_list = read_image(valid_images, config.val_img_path)
np.save('./mindrecord/valid_images_list', valid_images_list)

test_images_list = read_image(test_images, config.test_img_path)
np.save('./mindrecord/test_images_list', test_images_list)

### 1.5 生成MindRecord

In [10]:
generate_mindrecord(config.train_mindrecord_path, config.num_splits, train_images_list, train_questions_vec, train_answers_vec)
generate_mindrecord(config.valid_mindrecord_path, config.num_splits, valid_images_list, valid_questions_vec, valid_answers_vec)
generate_mindrecord(config.test_mindrecord_path,  config.num_splits, test_images_list,  test_questions_vec, test_answers_vec)

train
valid
test


## 2 加载数据

### 2.1 加载词典

In [4]:
# load dict
word_dict = np.load(config.dict_path, allow_pickle=True).item()
idx_word_dict = np.load(config.idx_word_dict_path, allow_pickle=True).item()

### 2.2 训练配置

In [5]:
model_name = 'baseline'

In [6]:
train_config = EasyDict({
	'model': model_name,
	'vocab_size': 10233,
	'batch_size': 128,
	'epoch_size': 3,
	'max_length': 25,
	'hidden_size': 1024,
	'lr': 1e-3,
	'momentum': 0.9,
	'early_stop': 100,
	# 'save_checkpoint_steps': 1279,
	'ckpt_save_path': './ckpt',
	'checkpoint_path': './ckpt/'+model_name+'.ckpt',
	# 'keep_checkpoint_max': 2,
})

### 2.3 生成数据集

In [7]:
# create dataset
train_dataset = generate_dataset(config.train_mindrecord_path, train_config.batch_size, train_config.epoch_size)
valid_dataset = generate_dataset(config.valid_mindrecord_path, train_config.batch_size, train_config.epoch_size)
test_dataset  = generate_dataset(config.test_mindrecord_path , train_config.batch_size, 1)

## 3 训练模型

### 3.1 创建模型

In [8]:
import mindspore.nn as nn
import mindspore.ops.operations as P
from utils.metric_utils import *
from utils.wrapper_utils import *
from utils.callback_utils import *

In [9]:
class Network(nn.Cell):
	def __init__(self, train_config):
		super(Network, self).__init__()
		self.reshape = P.Reshape()
		self.embedding = nn.Embedding(train_config.vocab_size, train_config.hidden_size)
		self.out = nn.Dense(train_config.hidden_size*train_config.max_length, train_config.vocab_size)
	def construct(self, images, questions):
		x = self.embedding(questions)
		x = x.reshape(x.shape[0], -1)
		x = self.out(x)
		return x

In [10]:
# 创建网络
network = Network(train_config)
network = LossAndAccWrapper(network, train_config)

### 3.2 开始训练

In [11]:
def train(network, train_dataset, valid_dataset, train_config):
	# 创建文件夹
	if not os.path.exists(train_config.ckpt_save_path):
		os.mkdir(train_config.ckpt_save_path)
	
	# 创建模型
	model = mindspore.Model(network, eval_network=network, metrics={'acc': DummyAccuracyMetric()})
	
	# 获取回调函数
	train_callbacks, _ = get_network_callbacks(model, train_dataset, valid_dataset, train_config)

	# 训练，保留最好模型
	model.train(train_config.epoch_size, train_dataset, callbacks=train_callbacks, dataset_sink_mode=True)


In [12]:
train(network, train_dataset, valid_dataset, train_config)

[WARNING] ME(90608:82636,MainProcess):2022-06-03-00:11:06.250.912 [mindspore\train\model.py:536] The CPU cannot support dataset sink mode currently.So the training process will be performed with dataset not sink.


epoch: 1 , step: 1 train loss = 9.235813 acc = 0.0
epoch: 1 , step: 2 train loss = 9.233626 acc = 0.0
epoch: 1 , step: 3 train loss = 9.23539 acc = 0.0
epoch: 1 , step: 4 train loss = 9.2367935 acc = 0.0
epoch: 1 , step: 5 train loss = 9.234498 acc = 0.0
epoch: 1 , step: 6 train loss = 9.236753 acc = 0.0
epoch: 1 , step: 7 train loss = 9.234799 acc = 0.0
epoch: 1 , step: 8 train loss = 9.235421 acc = 0.0
epoch: 1 , step: 9 train loss = 9.235122 acc = 0.0
epoch: 1 , step: 10 train loss = 9.236495 acc = 0.0
epoch: 1 , step: 11 train loss = 9.234311 acc = 0.0
epoch: 1 , step: 12 train loss = 9.234827 acc = 0.0
epoch: 1 , step: 13 train loss = 9.235803 acc = 0.0
epoch: 1 , step: 14 train loss = 9.234881 acc = 0.0
epoch: 1 , step: 15 train loss = 9.233698 acc = 0.0
epoch: 1 , step: 16 train loss = 9.2353115 acc = 0.0
epoch: 1 , step: 17 train loss = 9.235886 acc = 0.0
epoch: 1 , step: 18 train loss = 9.234626 acc = 0.0
epoch: 1 , step: 19 train loss = 9.237015 acc = 0.0
epoch: 1 , step: 20 

[WARNING] ME(90608:82636,MainProcess):2022-06-03-00:27:40.504.562 [mindspore\train\model.py:954] CPU cannot support dataset sink mode currently.So the evaluating process will be performed with dataset non-sink mode.


  valid loss = 9.236419 acc = 0.0
epoch: 2 , step: 958 train loss = 9.235848 acc = 0.0
epoch: 2 , step: 959 train loss = 9.23504 acc = 0.0
epoch: 2 , step: 960 train loss = 9.236348 acc = 0.0
epoch: 2 , step: 961 train loss = 9.235547 acc = 0.0
epoch: 2 , step: 962 train loss = 9.236148 acc = 0.0
epoch: 2 , step: 963 train loss = 9.236388 acc = 0.0
epoch: 2 , step: 964 train loss = 9.236352 acc = 0.0
epoch: 2 , step: 965 train loss = 9.235362 acc = 0.0
epoch: 2 , step: 966 train loss = 9.234002 acc = 0.0
epoch: 2 , step: 967 train loss = 9.23495 acc = 0.0
epoch: 2 , step: 968 train loss = 9.23493 acc = 0.0
epoch: 2 , step: 969 train loss = 9.235877 acc = 0.0
epoch: 2 , step: 970 train loss = 9.235649 acc = 0.0
epoch: 2 , step: 971 train loss = 9.235452 acc = 0.0
epoch: 2 , step: 972 train loss = 9.235906 acc = 0.0
epoch: 2 , step: 973 train loss = 9.237098 acc = 0.0
epoch: 2 , step: 974 train loss = 9.236225 acc = 0.0
epoch: 2 , step: 975 train loss = 9.235138 acc = 0.0
epoch: 2 , step

KeyboardInterrupt: 

## 4 测试模型

### 4.1 创建测试模型

In [ ]:
from mindspore.train.serialization import load_checkpoint

In [ ]:
network = Network(train_config)
load_checkpoint(train_config.checkpoint_path, net=network)
network = LossAndAccWrapper(network, train_config)

### 4.2 开始测试

In [ ]:
def test(network, test_dataset):
	# 创建测试模型
	model = mindspore.Model(network, metrics={'acc': DummyAccuracyMetric()})

	# 回调显示
	test_callbacks = [TestCallback()]

	# 生成结果
	model.eval(test_dataset, callbacks=test_callbacks, dataset_sink_mode=True)

In [ ]:
test(network, test_dataset)